In [1]:
import sympy as sp
from sympy import latex
from sympy.physics.mechanics import dynamicsymbols

import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d

import copernicusmarine
from scipy.integrate import cumulative_trapezoid as cumtrapz

import xarray as xr

In [2]:
t = dynamicsymbols._t


x, y, z, theta, phi = dynamicsymbols("x y z theta phi")  # x and y -> buoy coordinates

m_b, m_d, l, g = sp.symbols(
    "m_b m_d l g", positive=True
)  # mass buoy, mass drogue in [kg], wire length in [m], g in [m(s^2)]

k_b, k_d = sp.symbols("k_b k_d", positive=True)  # drag coefficients for buoy and drogue

U_b, V_b, U_d, V_d = sp.symbols(
    "U_b V_b U_d V_d", real=True
)  # surface horizontal velocities

# U_0, V_0 = sp.symbols('U_0 V_0', real = True) # surface horizontal velocities

The buoy is fixed to the water surface and initially has no possibility of lifting off or being pushed/pulled underwater (z = 0). The position of the drogue is described in spherical coordinates depending on the position of the buoy.

In [3]:
r_b = sp.Matrix([x, y, 0])  # buoy position

r = l * sp.Matrix(
    [
        sp.sin(theta) * sp.cos(phi),
        sp.sin(theta) * sp.sin(phi),
        sp.cos(theta),
    ]
)

r_d = r_b + r  # drogue position

display(r_d)

Matrix([
[l*sin(theta(t))*cos(phi(t)) + x(t)],
[l*sin(phi(t))*sin(theta(t)) + y(t)],
[                   l*cos(theta(t))]])

u_b and u_d represent the flow velocities at the water surface and at the height of the drogue. Later, the flow velocity below the water surface will be implemented as a function of z.

In [4]:
# velocities

v_b = r_b.diff(t)
v_d = r_d.diff(t)

u_b = sp.Matrix([U_b, V_b, 0])
u_d = sp.Matrix([U_d, V_d, 0])

F_b and F_d describe the drag forces on the buoy and the drogue.

In [5]:
def _mag(vec):
    return sp.sqrt(vec.dot(vec))


F_b = -k_b * _mag(v_b - u_b) * (v_b - u_b)  # drag buoy with ext. velocity
F_d = -k_d * _mag(v_d - u_d) * (v_d - u_d)  # drag drogue with ext. velocity

In [6]:
# lagrangian

T = sp.Rational(1, 2) * m_b * v_b.dot(v_b) + sp.Rational(1, 2) * m_d * v_d.dot(
    v_d
)  # kinetic energy

V = m_d * g * r_d[2]  # potential energy

L = T - V  # lagrangian

Defining $q$, $\dot{q}$ and $\ddot{q}$

In [7]:
q = sp.Matrix([x, y, theta, phi])
qd = q.diff(t)
qdd = qd.diff(t)

Generalized forces: $\left(\begin{array}{c}
        \frac{\partial r}{\partial \theta} \cdot F \\
        \frac{\partial r}{\partial \phi} \cdot F
        \end{array}\right)$

In [8]:
Q = sp.Matrix([r_b.diff(qi).dot(F_b) + r_d.diff(qi).dot(F_d) for qi in q])

Q = sp.simplify(Q)

Equations of motion:    

$\frac{\partial}{\partial t} (\frac{\partial L}{\partial \dot{q}_j}) - \frac{\partial L}{\partial q_j} - Q_j = 0$

In [9]:
eoms = sp.Matrix(
    [L.diff(qdj).diff(t) - L.diff(qj) - Qj for qj, qdj, Qj in zip(q, qd, Q)]
)
eoms = sp.simplify(eoms)

In [10]:
M, F = sp.linear_eq_to_matrix(eoms, list(qdd))

In [11]:
Ux, Uy = sp.symbols('Ux Uy', real = True)
subst = {
    x.diff(t): Ux,
    y.diff(t): Uy,
    
    theta.diff(t): 0,
    phi.diff(t): 0,

    x.diff(t, 2): 0,
    y.diff(t, 2): 0,
    theta.diff(t, 2): 0,
    phi.diff(t, 2): 0,  
}
eoms_steady_state = sp.simplify(eoms.subs(subst))
display(eoms_steady_state)

Matrix([
[                                              -k_b*(U_b - Ux)*sqrt((U_b - Ux)**2 + (Uy - V_b)**2) - k_d*(U_d - Ux)*sqrt((U_d - Ux)**2 + (Uy - V_d)**2)],
[                                               k_b*(Uy - V_b)*sqrt((U_b - Ux)**2 + (Uy - V_b)**2) + k_d*(Uy - V_d)*sqrt((U_d - Ux)**2 + (Uy - V_d)**2)],
[-l*(g*m_d*sin(theta(t)) + k_d*sqrt((U_d - Ux)**2 + (Uy - V_d)**2)*(U_d*cos(phi(t)) - Ux*cos(phi(t)) - Uy*sin(phi(t)) + V_d*sin(phi(t)))*cos(theta(t)))],
[                         k_d*l*sqrt((U_d - Ux)**2 + (Uy - V_d)**2)*(U_d*sin(phi(t)) - Ux*sin(phi(t)) + Uy*cos(phi(t)) - V_d*cos(phi(t)))*sin(theta(t))]])

In [12]:
parameter = {
    k_b: 3.0,
    k_d: 2.0,
    U_b: 1.0, V_b: 1.5,
    U_d: 2.0, V_d: -0.5,
    g: 9.81,
    l: 3.0,
    m_d: 1.0,
}

th, ph = sp.symbols('th ph', real = True)

equations_with_params = eoms_steady_state.subs(parameter).subs({theta:th, phi:ph})

In [13]:
equations = list(equations_with_params)
sol = sp.nsolve(equations, (Ux, Uy, th, ph), (0.0, 0.0, 0.2, 1.0))
sol

Matrix([
[    1.44948974278318],
[   0.601020514433644],
[3.34882109737048e-27],
[   0.463647609000806]])

In [14]:
eqs = list(eoms_steady_state.subs({theta:th, phi:ph}))
sol_a = sp.solve(eqs, (Ux, Uy, th, ph))

KeyboardInterrupt: 